<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84%EB%90%9C_H_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_epoch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 15.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=ff23cad8b89ab4044c108285ca162a9cbd9bbb62fe3f238ca0693b01bd06bb4b
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 16.1 MB/s 
     |████████████████████████████████| 769 kB 10.6 MB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 
     |████████████████████████████████| 3.0 MB 52.9 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-vt73exz7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_G_inc.csv		   df_P_inc.csv
'Colab Notebooks'	 df_H_inc_노이.csv	  'My Drive'
 C_ori_model.pt		 df_J_inc_again_노이.csv   Q_inc_model.pt
'답안 작성용 파일.csv'	 df_J_inc.csv		   spell_check
 df_C_inc.csv		 df_J_inc_dain.csv
'# df_G_inc.csv'	 df_K_inc.csv


In [4]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/df_H_inc_노이.csv')

## 데이터 전처리

In [5]:
df['digit_3'].value_counts()

493.0    49115
492.0    39176
529.0     5605
512.0     4500
491.0     2900
501.0     2810
494.0     2134
502.0     1700
511.0     1600
521.0     1591
495.0     1500
Name: digit_3, dtype: int64

In [6]:
df.loc[(df['digit_3'] == 491), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 492), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 493), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 494), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 495), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 501), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 502), 'digit_3'] = 6  
df.loc[(df['digit_3'] == 511), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 512), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 521), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 529), 'digit_3'] = 10  

In [7]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(int(label))

    data_list.append(data)

print(data_list[:5])

[['개인택시로 일반인을 대상으로 승객 운송서비스', 1], ['개인택시로 일반인을 대상으로 승객 운송서비스', 1], ['개인택시로 일반인을 대상으로 승객 운송서비스', 1], ['개인택시로 일반인을 대상으로 승객 운송서비스', 1], ['개인택시로 일반인을 대상으로 승객 운송서비스', 1]]


## Train data & test data

In [8]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [11]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [13]:
data_train[0]

(array([   2,  841, 7941, 6241, 7151, 6079,  994, 7095, 3490, 7078, 5112,
        6241, 3516, 6555,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [17]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1584 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.5764613151550293 train acc 0.03125
epoch 1 batch id 201 loss 0.48009222745895386 train acc 0.5998134328358209
epoch 1 batch id 401 loss 0.17420068383216858 train acc 0.7625467581047382
epoch 1 batch id 601 loss 0.07144946604967117 train acc 0.8306988352745425
epoch 1 batch id 801 loss 0.09385748952627182 train acc 0.8668851435705368
epoch 1 batch id 1001 loss 0.08456522971391678 train acc 0.8886582167832168
epoch 1 batch id 1201 loss 0.03580285981297493 train acc 0.9037260616153205
epoch 1 batch id 1401 loss 0.11044537276029587 train acc 0.9145476445396146
epoch 1 train acc 0.922406056301653


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9912997159090909


  0%|          | 0/1584 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0029166422318667173 train acc 1.0
epoch 5 batch id 201 loss 0.004724834579974413 train acc 0.9952580845771144
epoch 5 batch id 401 loss 0.0009501157910563052 train acc 0.9952462593516209
epoch 5 batch id 601 loss 0.0031432320829480886 train acc 0.9956842762063228
epoch 5 batch id 801 loss 0.0013149427250027657 train acc 0.9954744069912609
epoch 5 batch id 1001 loss 0.0009858842240646482 train acc 0.9955669330669331
epoch 5 batch id 1201 loss 0.05409422144293785 train acc 0.9955375728559533
epoch 5 batch id 1401 loss 0.0037063974887132645 train acc 0.995494289793005
epoch 5 train acc 0.9954706869834711


  0%|          | 0/176 [00:00<?, ?it/s]

epoch 5 test acc 0.9909446022727273


In [18]:
#학습 모델 저장
torch.save(model, 'H_inc_model_epoch5.pt')